# Analysis of Productivity and Population Data

#### We begin with two raw datasets, one from the Bureau of Labor Statistics (BLS) covering producitivity by sector and quarter (period) from 1995 through 2023, and another from Data USA providing population data for the United States of America between 2013 and 2021.

Our BLS data is available on S3 at the following links:
- [pr.class](https://790890014576-rearc-data-quest-bucket.s3.amazonaws.com/data/bls/pr.class)
- [pr.contacts](https://790890014576-rearc-data-quest-bucket.s3.amazonaws.com/data/bls/pr.contacts)
- [pr.data.0.Current](https://790890014576-rearc-data-quest-bucket.s3.amazonaws.com/data/bls/pr.data.0.Current)
- [pr.data.1.AllData](https://790890014576-rearc-data-quest-bucket.s3.amazonaws.com/data/bls/pr.data.1.AllData)
- [pr.duration](https://790890014576-rearc-data-quest-bucket.s3.amazonaws.com/data/bls/pr.duration)
- [pr.footnote](https://790890014576-rearc-data-quest-bucket.s3.amazonaws.com/data/bls/pr.footnote)
- [pr.measure](https://790890014576-rearc-data-quest-bucket.s3.amazonaws.com/data/bls/pr.measure)
- [pr.period](https://790890014576-rearc-data-quest-bucket.s3.amazonaws.com/data/bls/pr.period)
- [pr.seasonal](https://790890014576-rearc-data-quest-bucket.s3.amazonaws.com/data/bls/pr.seasonal)
- [pr.sector](https://790890014576-rearc-data-quest-bucket.s3.amazonaws.com/data/bls/pr.sector)
- [pr.series](https://790890014576-rearc-data-quest-bucket.s3.amazonaws.com/data/bls/pr.series)
- [pr.txt](https://790890014576-rearc-data-quest-bucket.s3.amazonaws.com/data/bls/pr.txt)

In [1]:
### Imports
import os
import io
import json
import requests
import pandas as pd

In [2]:
### Global Variable Setting

bls_data_url = "https://download.bls.gov/pub/time.series/pr/pr.data.0.Current"
pop_data_url = "https://datausa.io/api/data?drilldowns=Nation&measures=Population"
headers = {"User-Agent": "seancowens13@gmail.com"} # If we don't include this we will be flagged as a bot by the data providers

In [3]:
### GET data

bls_response = requests.get(url=bls_data_url, headers=headers)
bls_response.raise_for_status()

pop_response = requests.get(url=pop_data_url, headers=headers)
pop_response.raise_for_status()

#### We now have two successful responses (i.e. HTTP status code between 200 and 300). Let's try transforming the data to a DataFrame right away and see what happens.

In [4]:
bls_df = pd.read_csv(bls_response.content)

TypeError: Expected file path name or file-like object, got <class 'bytes'> type

In [5]:
pop_df = pd.DataFrame(pop_response.json())

ValueError: All arrays must be of the same length

#### Two errors. Not a great start unfortunately, but we can't expect public datasets to be perfectly formatted for all of our needs. Let's tackle them one at a time to determine what needs to be done.

1. BLS: The data is meant to be in a csv format, however, our request.Response.content is all bytes. We'll need to decode the response content, stream it to an in-memory text buffer, and pass this into our `read_csv` DataFrame constructor. As we do this, we'll get back a DF with a single column and a lot of `\t` throughout. The fix here is simple, the data is tab delimited and we simply note this as kwarg in `read_csv`.
2. Population: Upon further inspection, there are two top level keys, `data` and `source`. However, each of these has different data in their sub-arrays, causing Pandas to return the error `ValueError: All arrays must be of the same length`. We're only interested in the `data` portion of the `response.content` anyways, so we can scrap the `source` key and try our DataFrame constructor once more.

In [6]:
bls_df = pd.read_csv(io.StringIO(bls_response.content.decode("utf-8")), delimiter="\t")

In [7]:
bls_df.shape

(35466, 5)

In [8]:
pop_df = pd.DataFrame(pop_response.json()["data"])

In [9]:
pop_df.shape

(9, 6)

#### So far so good. Let's get started with some simple aggregated statistics on the population data. First things first, we should inspect the DF to ensure there is nothing strange about the column names or the data itself.

In [10]:
print(pop_df.columns)
pop_df.head()

Index(['ID Nation', 'Nation', 'ID Year', 'Year', 'Population', 'Slug Nation'], dtype='object')


,ID Nation,Nation,ID Year,Year,Population,Slug Nation
0,01000US,United States,2021,2021,329725481,united-states
1,01000US,United States,2020,2020,326569308,united-states
2,01000US,United States,2019,2019,324697795,united-states
3,01000US,United States,2018,2018,322903030,united-states
4,01000US,United States,2017,2017,321004407,united-states


#### Looks pretty good overall! To make things a little more consistent, let's lowercase all columns names. Aside from that we should all set to calculate our mean and standard deviation from 2013 to 2018.

In [11]:
pop_df.rename(columns=lambda x: x.lower(), inplace=True)
pop_df

,id nation,nation,id year,year,population,slug nation
0,01000US,United States,2021,2021,329725481,united-states
1,01000US,United States,2020,2020,326569308,united-states
2,01000US,United States,2019,2019,324697795,united-states
3,01000US,United States,2018,2018,322903030,united-states
4,01000US,United States,2017,2017,321004407,united-states
5,01000US,United States,2016,2016,318558162,united-states
6,01000US,United States,2015,2015,316515021,united-states
7,01000US,United States,2014,2014,314107084,united-states
8,01000US,United States,2013,2013,311536594,united-states


#### First, we should narrow our selection to the years of interest before performing any calculations.

In [12]:
years_of_interest_df = pop_df[pop_df["year"].between(2012, 2019, inclusive="neither")]
years_of_interest_df

TypeError: '>' not supported between instances of 'str' and 'int'

In [13]:
pop_df.dtypes

id nation      object
nation         object
id year         int64
year           object
population      int64
slug nation    object
dtype: object

#### It appears our year column has been cast as a string `year` has the data type `object`, we should coerce this to a numerical data type to work with in the future.

In [14]:
pop_df["year"] = pop_df["year"].astype(int)
pop_df.dtypes

id nation      object
nation         object
id year         int64
year            int64
population      int64
slug nation    object
dtype: object

#### Now we can retry our subselection!

In [15]:
years_of_interest_df = pop_df[pop_df["year"].between(2012, 2019, inclusive="neither")]
years_of_interest_df

,id nation,nation,id year,year,population,slug nation
3,01000US,United States,2018,2018,322903030,united-states
4,01000US,United States,2017,2017,321004407,united-states
5,01000US,United States,2016,2016,318558162,united-states
6,01000US,United States,2015,2015,316515021,united-states
7,01000US,United States,2014,2014,314107084,united-states
8,01000US,United States,2013,2013,311536594,united-states


In [16]:
mean_usa_population = int(years_of_interest_df["population"].mean())
mean_usa_population

317437383

In [17]:
std_dev_usa_population = int(years_of_interest_df["population"].std())
std_dev_usa_population

4257089

#### Great! We were able to calculate the mean and standard deviation for the USA Population across the years 2013 through 2018. Let's move onto the BLS data and some more complex queries.

In [18]:
print(bls_df.columns)
bls_df.head()

Index(['series_id        ', 'year', 'period', '       value',
       'footnote_codes'],
      dtype='object')


,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.6,NaN
1,PRS30006011,1995,Q02,2.1,NaN
2,PRS30006011,1995,Q03,0.9,NaN
3,PRS30006011,1995,Q04,0.1,NaN
4,PRS30006011,1995,Q05,1.4,NaN


#### Yikes! Right away, it's clear this one needs some clean up. The columns have tons of whitespace peppered throughout, and if the columns are suffering from this issue, it's likely the data is too. Let's take some remediation steps to clear this up.

In [19]:
bls_df.rename(columns=lambda x: x.strip(), inplace=True)
bls_df = bls_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x) # For any data which is non-numerical, strip whitespace.
print(bls_df.columns)
bls_df.head()

Index(['series_id', 'year', 'period', 'value', 'footnote_codes'], dtype='object')


,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.6,NaN
1,PRS30006011,1995,Q02,2.1,NaN
2,PRS30006011,1995,Q03,0.9,NaN
3,PRS30006011,1995,Q04,0.1,NaN
4,PRS30006011,1995,Q05,1.4,NaN


#### Much better! Now we can move onto performing our query: For each `series_id`, find the best year - the year with the max/largest sum of `value` for all quarters in that year. The output should generate a report with the series_id, the best year of that series_id, and the summed value for that year.

#### This means our final output should be a condensed DF with a row count equal to the count of unique series_id in the raw DF: 282.

In [20]:
len(bls_df["series_id"].unique())

282

#### Below we perform the first aggregation, sum the values by year and by series so we are left with a DF containing the sum of all quarters for that year per series. Now we can perform a filter on this DF based on series_id to select only the year with the highest value for each series.

In [21]:
summed_by_year_series_df = bls_df[["series_id", "year", "value"]].groupby(["series_id", "year"], as_index=False).sum()
summed_by_year_series_df

,series_id,year,value
0,PRS30006011,1995,7.100
1,PRS30006011,1996,-0.500
2,PRS30006011,1997,4.400
3,PRS30006011,1998,4.200
4,PRS30006011,1999,-7.700
...,...,...,...
8083,PRS88003203,2019,519.449
8084,PRS88003203,2020,520.840
8085,PRS88003203,2021,530.355
8086,PRS88003203,2022,568.533


#### Below we have our output! We can confirm that we indeed have 282 unique series values in our DF below, along with the greatest value for each given series. This aggregation performed the group by on the series id (looking at all year, value combinations for a series id), selected the index of the maximum value, then using `.loc` we accessed the location of each record in `summed_by_year_series_df` and returned it to our final DataFrame.

In [22]:
greatest_year_by_series = summed_by_year_series_df.loc[summed_by_year_series_df.groupby("series_id")["value"].idxmax()].reset_index(drop=True)
greatest_year_by_series

,series_id,year,value
0,PRS30006011,2022,20.500
1,PRS30006012,2022,17.100
2,PRS30006013,1998,704.125
3,PRS30006021,2010,17.600
4,PRS30006022,2010,12.500
...,...,...,...
277,PRS88003192,2002,282.800
278,PRS88003193,2022,765.817
279,PRS88003201,2022,36.100
280,PRS88003202,2022,28.900


In [23]:
len(greatest_year_by_series["series_id"].unique())

282

#### Finally, let's join these two datasets. Given the population dataset is a subset of the time period covered by the BLS dataset, we will `left join` population onto BLS, with our join key being `year`.

In [24]:
pop_bls_df = pd.merge(bls_df[["series_id", "year", "period", "value"]], pop_df[["year", "population"]], how="left", on=["year"])
pop_bls_df

,series_id,year,period,value,population
0,PRS30006011,1995,Q01,2.600,NaN
1,PRS30006011,1995,Q02,2.100,NaN
2,PRS30006011,1995,Q03,0.900,NaN
3,PRS30006011,1995,Q04,0.100,NaN
4,PRS30006011,1995,Q05,1.400,NaN
...,...,...,...,...,...
35461,PRS88003203,2022,Q04,115.039,NaN
35462,PRS88003203,2022,Q05,113.713,NaN
35463,PRS88003203,2023,Q01,116.214,NaN
35464,PRS88003203,2023,Q02,116.953,NaN


#### On the face of it, this does appear too interesting, give we don't have population data for the head nor tail of the DF. Let's confirm our join was successful by narrowing down to those rows where population is not null.

In [25]:
pop_bls_df[pop_bls_df["population"].notnull()]

,series_id,year,period,value,population
90,PRS30006011,2013,Q01,1.400,311536594.0
91,PRS30006011,2013,Q02,0.400,311536594.0
92,PRS30006011,2013,Q03,0.300,311536594.0
93,PRS30006011,2013,Q04,0.700,311536594.0
94,PRS30006011,2013,Q05,0.700,311536594.0
...,...,...,...,...,...
35453,PRS88003203,2021,Q01,104.239,329725481.0
35454,PRS88003203,2021,Q02,104.345,329725481.0
35455,PRS88003203,2021,Q03,106.517,329725481.0
35456,PRS88003203,2021,Q04,109.157,329725481.0


#### There we go! Now we are confident our data has been merged successfully.

## And that's a wrap! Thank you for your time and consideration. I look forward to your feedback.